In [1]:
import os
import sys
import warnings

sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings('ignore')

In [2]:
import autograd.numpy as np
from autograd import grad 

import geomstats.backend as gs
from geomstats.geometry.stiefel import StiefelCanonicalMetric

import pymanopt
from pymanopt import Problem
from pymanopt.manifolds.stiefel import Stiefel
from pymanopt.solvers import SteepestDescent, TrustRegions

INFO: Using numpy backend


In [3]:
eig_val = 3
c1 = gs.eye(eig_val)
print(type(c1))
print(c1)

<class 'numpy.ndarray'>
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [4]:
metric = StiefelCanonicalMetric(n=eig_val, p=eig_val)

In [5]:
def sqdist(X):
    print(type(X))
    return metric.squared_dist(X._value, c1)

grad_sqdist = grad(sqdist)
grad_sqdist(c1)  

<class 'autograd.numpy.numpy_boxes.ArrayBox'>


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

# Proof of concept

Consider the function: $f: St(p, p) \rightarrow \mathbb{R}$ defined by:
$$f(X) = d^2(X, C_1)$$
where $d^2$ is the squared geodesic distance associated with the canonical Riemannian metric on the Stiefel manifold $St(p, p)$.

The gradient of $f$ at $X \in St(p, p)$ is an element of $T_X(St(p, p))$ and defined as:
$$\nabla_Xf = - 2 . \text{Log}_X(C_1),$$
where Log is the Riemannian logarithm associated with the canonical Riemannian metric on the Stiefel manifold $St(p, p)$.

We can provide this gradient in closed form when minimizing the function $f$ using `pymanopt`, as in the example below.

In [6]:
# (1) Instantiate a manifold
manifold = Stiefel(eig_val, eig_val)

# (2) Define the cost function (here using autograd.numpy)

@pymanopt.function.Autograd
def cost(X):
    if isinstance(X, np.ndarray):
        return metric.squared_dist(X, c1)
    else:
        return metric.squared_dist(X._value, c1)
       
@pymanopt.function.Autograd
def custom_grad(X):
    if isinstance(X, np.ndarray):
        return -2. * metric.log(base_point=X, point=c1)
    else:
        return -2. * metric.log(base_point=X._value, point=c1)

problem = Problem(manifold=manifold, cost=cost, grad=custom_grad, verbosity=2)

# (3) Instantiate a Pymanopt solver
solver = SteepestDescent(logverbosity=2)

# let Pymanopt do the rest
res = solver.solve(problem, x=manifold.rand())
Xopt = res[0]

#print(Xopt.shape)
#display(res)

 iter		   cost val	    grad. norm
    1	+5.7397101018431238e+00	6.77625861e+00
    2	+3.1547241959023098e+00	5.02372308e+00
    3	+6.4726581459688703e-01	2.27554972e+00
    4	+2.3397329703573749e-01	1.36813244e+00
    5	+5.4526899713703406e-02	6.60465894e-01
    6	+1.9787028762498864e-02	3.97864588e-01
    7	+1.0002160451138860e-02	2.82873264e-01
    8	+7.8905132825875594e-06	7.94506805e-03
    9	+4.4218437209285152e-07	1.88081764e-03
   10	+1.2331537572125891e-09	9.93238645e-05
   11	+1.9415446392991809e-10	3.94111115e-05
   12	+2.2161637619295751e-11	1.33151455e-05
   13	+1.9590693073327987e-11	1.25190073e-05
   14	+1.4786196511006190e-11	1.08761010e-05
   15	+6.7383338360276426e-12	7.34211623e-06
   16	+2.5449449056077943e-13	1.42686927e-06
   17	+8.9291487757510406e-14	8.45181579e-07
Terminated - min grad norm reached after 17 iterations, 0.32 seconds.



In [7]:
print(metric.squared_dist(Xopt, c1))

6.453648507536602e-14


In [8]:
print(metric.squared_dist(manifold.rand(), c1))

5.199839258212869


# General case with the cost of Eq. (8)

In the general case, the function that we seek to minimize is $F: St(p, p)^{n-1} \rightarrow \mathbb{R}$ defined on $n-1$ copies of the Stiefel manifold $St(p, p) \times ... \times St(p, p)=St(p, p)^{n-1}$ as:
$$F(\mathbf{C}) = F(C_2, ..., C_n) = \sum_{i, j \in \mathcal{E}} d^2(C_i^{-1}.C_j, C_{ij}).$$

We want to derive $F$ as a function of $\mathbf{C} \in St(p, p)^{n-1}$, where $\mathbf{C}$ has shape $(n-1) \times (n-1)p$ that is taking values in $\mathbb{R}$. The gradient of $F$ at $\mathbf{C}_0$, written $\nabla_{\mathbf{C}_0}F$ can be represented as matrix of shape $(n-1)p \times (n-1)$ as:
$$\nabla_{\mathbf{C}_0}F 
= \frac{\partial F}{\partial \mathbf{C}}(\mathbf{C}_0)
= \begin{pmatrix}
\frac{\partial F}{\partial C_2}(\mathbf{C}_0)\\
...\\
\frac{\partial F}{\partial C_n}(\mathbf{C}_0)
\end{pmatrix}
$$

Thus, we compute each $\frac{\partial F}{\partial C_k}(\mathbf{C}_0)$, for $k \in \{2, ..., n\}.$

We have:
$$\begin{align*}
\frac{\partial F}{\partial C_k}(\mathbf{C}_0)
&= \frac{\partial}{\partial C_k} \left(
    \sum_{i, j \in \mathcal{E}} d^2(C_i^{-1}.C_j, C_{ij})\right)(
        \mathbf{C}_0)\\
&= \frac{\partial}{\partial C_k}\left(
    \sum_{j \text{ linked to } k }d^2(C_k^{-1}.C_j, C_{kj}) 
    + \sum_{i \text{ linked to } k } d^2(C_i^{-1}.C_k, C_{ki})\right)(
        \mathbf{C}_0)\\
\end{align*}$$

Thus, we need to define the derivates of the functions $g_j: St(p, p)\rightarrow \mathbb{R}$ defined as:
$$g_j(C_k) = d^2(C_k^{-1}.C_j, C_{kj},$$

and of the functions $h_i: St(p, p)\rightarrow \mathbb{R}$ defined as:
$$h_i(C_k) = d^2(C_i^{-1}.C_k, C_{ki}).$$

The derivative of the $h_i$ at $\mathbf{C}_0$ is defined as:
$$\nabla_{\mathbf{C}_0}h_i 
= - 2 . (C_{i0}^{-T}) . \text{Log}_{C_{i0}^{-1}C_{k0}}C_{ki}
= - 2 . C_{i0} . \text{Log}_{C_{i0}^{-1}C_{k0}}C_{ki}$$

See:
- https://math.stackexchange.com/questions/1377457/how-to-get-the-derivative-of-a-matrix-function
- and the book "The matrix cookbook" available freely online.